In [ ]:
import os
import collections
import re
from typing import List, Dict
from openai import OpenAI

# ---------- Config ----------
MODEL = "gpt-4o-mini"         # pick any chat model you have access to
QUESTION = "Which element has the chemical symbol 'W'?"
TEMPS = [0.2, 0.5, 0.8, 1.2]  # temperatures to test
SAMPLES_PER_TEMP = 7          # how many samples to draw at each temperature
MAX_TOKENS = 64               # response length cap
os.environ["OPENAI_API_KEY"] =  "set key hide from repo"  # set your OpenAI API key
# ----------------------------

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def normalize(text: str) -> str:
    """Light normalization + simple synonym mapping example."""
    t = text.strip().lower()
    t = re.sub(r"[^a-z \-]", "", t)
    # Treat 'wolfram' as synonym for 'tungsten'
    if "wolfram" in t:
        return "tungsten"
    return t

def one_sample(question: str, temperature: float, seed: int | None = None) -> str:
    """
    Get a single sampled answer at a given temperature.
    Note: 'seed' is optional; many Chat Completions models accept it.
    If unsupported for your model, remove the 'seed' argument below.
    """
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": question}],
        temperature=temperature,
        max_tokens=MAX_TOKENS,
        n=1,
        seed=seed,  # comment out if your model rejects this param
    )
    return resp.choices[0].message.content.strip()

def sample_many(question: str, temperature: float, k: int) -> List[str]:
    """Draw k independent samples at a fixed temperature."""
    answers = []
    for i in range(k):
        txt = one_sample(question, temperature, seed=i)  # vary seed for reproducibility
        answers.append(txt)
    return answers

def majority_vote(answers: List[str]) -> Dict:
    """Return raw answers, normalized votes, and the winner."""
    normed = [normalize(a) for a in answers]
    tally = collections.Counter(normed)
    winner, count = tally.most_common(1)[0]
    return {
        "raw": answers,
        "normalized": normed,
        "tally": tally,
        "winner": winner,
        "votes": count,
    }

def run(question: str, temps: List[float], k: int):
    print(f"\nQuestion: {question}\nModel: {MODEL}\nSamples per temperature: {k}\n")
    for T in temps:
        answers = sample_many(question, T, k)
        result = majority_vote(answers)
        print(f"--- Temperature {T:.2f} ---")
        print(f"Majority: {result['winner']}  ({result['votes']}/{k} votes)")
        # (optional) uncomment to see all raw answers:
        # for i, a in enumerate(result["raw"], 1):
        #     print(f"{i}. {a}")
        print()

if __name__ == "__main__":
    run(QUESTION, TEMPS, SAMPLES_PER_TEMP)



Question: Which element has the chemical symbol 'W'?
Model: gpt-4o-mini
Samples per temperature: 7

--- Temperature 0.20 ---
Majority: the chemical symbol w stands for tungsten  (3/7 votes)

--- Temperature 0.50 ---
Majority: the chemical symbol w stands for tungsten  (3/7 votes)

--- Temperature 0.80 ---
Majority: the element with the chemical symbol w is tungsten  (4/7 votes)

--- Temperature 1.20 ---
Majority: the element with the chemical symbol w is tungsten  (4/7 votes)

